# 第 4 章: Apache Spark - 3

本ノートブックでは、「基本的なIceberg機能の利用」および「高度なIceberg機能の利用」節で紹介されている例を実行できます。Spark Structured Streaming を Iceberg で利用する方法については、本ノートブック後半の「Spark Structured Streaming での Iceberg 利用」パートを確認してください。

In [ ]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
CATALOG = "my_catalog"
CATALOG_URL = "http://server:8181/"
S3_ENDPOINT = "http://minio:9000"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.8.1"

### SparkSession オブジェクトを初期化する

In [ ]:
spark = (
    SparkSession.builder
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}")
        .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
        .config(f"spark.sql.catalog.{CATALOG}.type", "rest")
        .config(f"spark.sql.catalog.{CATALOG}.uri", CATALOG_URL)
        .config(f"spark.sql.catalog.{CATALOG}.s3.endpoint", S3_ENDPOINT)
        .config(f"spark.sql.catalog.{CATALOG}.view-endpoints-supported", "true")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.defaultCatalog", "my_catalog")
        .getOrCreate()
)

In [ ]:
%sql spark

#### (Optional) データベースの作成
データベースを作成していない場合、以下のセルを実行してください。既にデータベースが存在する場合は、本ステップにつきましてはスキップしてください。

In [ ]:
%%sql
CREATE DATABASE IF NOT EXISTS db

## 基本的なIceberg機能の利用
### テーブルの作成

In [ ]:
%%sql
CREATE OR REPLACE TABLE db.sales_iceberg (
    product_name string,
    price decimal(10, 2),
    customer_id bigint,
    order_id string,
    datetime timestamp,
    category string) 
USING iceberg

#### テーブルロケーションの設定

In [ ]:
%%sql
CREATE TABLE db.sales_iceberg (
    product_name string,
    price decimal(10, 2),
    customer_id bigint,
    order_id string,
    datetime timestamp,
    category string) 
USING iceberg
LOCATION 's3://amzn-s3-demo-bucket/custom-path'

#### テーブルプロパティの設定

In [ ]:
%%sql
CREATE TABLE db.sales_iceberg (
    product_name string,
    price decimal(10, 2),
    customer_id bigint,
    order_id string,
    datetime timestamp,
    category string)
USING iceberg
TBLPROPERTIES (
    'write.metadata.compression-codec'='gzip')

#### テーブルパーティションの設定

In [ ]:
%%sql
CREATE TABLE db.sales_iceberg (
    product_name string,
    price decimal(10, 2),
    customer_id bigint,
    order_id string,
    datetime timestamp,
    category string)
USING iceberg
PARTITIONED BY (category, year(datetime))

### テーブルデータの読み込み
テーブルレコードの準備: 

In [ ]:
%%sql
INSERT INTO db.sales_iceberg VALUES
    ('tomato juice', 2.00, 1698, 'DRE8DLTFNX0MLCE8DLTFNX0MLC', TIMESTAMP '2023-07-18T02:20:58Z', 'drink'),
    ('cocoa', 2.00, 1652, 'DR1UNFHET81UNFHET8', TIMESTAMP '2024-08-26T11:36:48Z', 'drink'),
    ('espresso', 2.00, 1037, 'DRBFZUJWPZ9SRABFZUJWPZ9SRA', TIMESTAMP '2024-04-19T12:17:22Z', 'drink'),
    ('broccoli', 1.00, 3092, 'GRK0L8ZQK0L8ZQ', TIMESTAMP '2023-03-22T18:48:04Z', 'grocery'),
    ('nutmeg', 1.00, 3512, 'GR15U0LKA15U0LKA', TIMESTAMP '2024-02-27T15:13:31Z', 'grocery')

テーブルデータを読む:

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg

### データの書き込み
`INSERT INTO` によるデータの追加

In [ ]:
%%sql
INSERT INTO db.sales_iceberg VALUES
    ('tomato juice', 2.00, 1698, 'DRE8DLTFNX0MLCE8DLTFNX0MLC', TIMESTAMP '2023-07-18T02:20:58Z', 'drink'),
    ('cocoa', 2.00, 1652, 'DR1UNFHET81UNFHET8', TIMESTAMP '2024-08-26T11:36:48Z', 'drink'),
    ('espresso', 2.00, 1037, 'DRBFZUJWPZ9SRABFZUJWPZ9SRA', TIMESTAMP '2024-04-19T12:17:22Z', 'drink'),
    ('broccoli', 1.00, 3092, 'GRK0L8ZQK0L8ZQ', TIMESTAMP '2023-03-22T18:48:04Z', 'grocery'),
    ('nutmeg', 1.00, 3512, 'GR15U0LKA15U0LKA', TIMESTAMP '2024-02-27T15:13:31Z', 'grocery')

`UPDATE` によるデータの更新

In [ ]:
%%sql
UPDATE db.sales_iceberg 
SET product_name = 'white mocha',price = 4.0, datetime = CURRENT_TIMESTAMP
WHERE product_name = 'espresso'

`DELETE` によるデータの削除

In [ ]:
%%sql
DELETE FROM db.sales_iceberg WHERE year(datetime) < 2024

## 高度なIceberg機能の利用

### CTAS によるテーブルの作成

In [ ]:
%%sql
CREATE TABLE db.sales_iceberg_ctas
USING iceberg
AS SELECT * FROM db.sales_iceberg

テーブル作成後、テーブルデータについて確認

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_ctas

### スキーマ進化

スキーマ進化実行前に `sales_iceberg` テーブルのカラムを確認

In [ ]:
%%sql
DESCRIBE db.sales_iceberg

In [ ]:
%%sql
ALTER TABLE db.sales_iceberg ADD COLUMN description string AFTER product_name

In [ ]:
%%sql
ALTER TABLE db.sales_iceberg ADD COLUMN description string

### パーティション進化

In [ ]:
%%sql
ALTER TABLE db.sales_iceberg ADD PARTITION FIELD category

### ビュー

In [ ]:
%%sql
CREATE OR REPLACE VIEW db.sales_iceberg_analysis_view AS 
SELECT category, sum(price) as total_sales, count(*) as count_by_year, year(datetime) as year
FROM db.sales_iceberg 
GROUP BY category, year

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_analysis_view ORDER BY year DESC, category ASC

### タイムトラベルクエリ

テーブルに新たなデータを追加する

In [ ]:
%%sql
INSERT INTO db.sales_iceberg VALUES
    ('broccoli', 1.00, 3092, 'GRK0L8ZQK0L8ZQ', TIMESTAMP '2023-03-22T18:48:04Z', 'grocery'),
    ('nutmeg', 1.00, 3512, 'GR15U0LKA15U0LKA', TIMESTAMP '2024-02-27T15:13:31Z', 'grocery')

現在のテーブルデータを確認する

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg TIMESTAMP AS OF '<INSERT する前の時間を入力 (例: 2025-03-28 10:00:00)>'

### メタデータテーブルクエリ

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg.history

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg.snapshots

### MERGE INTO によるデータの更新と追加

In [ ]:
%%sql
MERGE INTO db.sales_iceberg_w t
USING db.sales_logs s
ON t.order_id = s.order_id
WHEN MATCHED THEN 
    UPDATE SET 
        t.product_name=s.product_name, 
        t.price=s.price, 
        t.datetime=s.datetime
WHEN NOT MATCHED THEN INSERT *

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_w ORDER BY category, product_name

#### WHEN NOT MATCHED BY SOURCE
sales_logs へ新たに 2 レコードを追加します。`mocha` は古いレコーとがたまたま流れてきてしまったことを想定しています。

In [ ]:
%%sql
INSERT INTO db.sales_logs VALUES
    ('mocha', 4.00, 1652, 'DR1UNFHET81UNFHET8', TIMESTAMP '2013-11-26T12:49:43Z', 'drink'),
    ('egg', 1.00, 3176, 'GRVQARCD6COVQARCD6CO', TIMESTAMP '2025-02-10 11:15:31', 'grocery');

In [ ]:
%%sql
SELECT * FROM db.sales_logs ORDER BY category, product_name

In [ ]:
%%sql
MERGE INTO db.sales_iceberg_w t
USING db.sales_logs s
ON t.order_id = s.order_id
WHEN MATCHED AND t.datetime < s.datetime THEN 
    UPDATE SET 
        t.product_name=s.product_name, 
        t.price=s.price, 
        t.datetime=s.datetime
WHEN NOT MATCHED THEN INSERT *
WHEN NOT MATCHED BY SOURCE THEN DELETE

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_w ORDER BY category, product_name

---
## Spark Structured Streaming での Iceberg 利用
### (Optional) 事前準備

まず SparkSession を初期化します。既に初期化してしまっている場合は、本カーネルを restart してください。

In [ ]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
CATALOG = "my_catalog"
CATALOG_URL = "http://server:8181/"
S3_ENDPOINT = "http://minio:9000"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.8.1"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"

In [ ]:
spark = (
    SparkSession.builder
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION},org.apache.hadoop:hadoop-aws:3.2.4,org.apache.hadoop:hadoop-client:3.2.4")
        .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
        .config(f"spark.sql.catalog.{CATALOG}.type", "rest")
        .config(f"spark.sql.catalog.{CATALOG}.uri", CATALOG_URL)
        .config(f"spark.sql.catalog.{CATALOG}.s3.endpoint", S3_ENDPOINT)
        .config(f"spark.sql.catalog.{CATALOG}.view-endpoints-supported", "true")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.defaultCatalog", "my_catalog")
        .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
        .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
        .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT)
        .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
        .getOrCreate()
)

In [ ]:
%sql spark

事前に `sales_iceberg` と `sales_iceberg_analysis` Iceberg テーブルが必要なため、テーブルが存在しない場合は作成しておきます。

In [ ]:
%%sql
CREATE OR REPLACE TABLE db.sales_iceberg (
    product_name string,
    price decimal(10, 2),
    customer_id bigint,
    order_id string,
    datetime timestamp,
    category string) 
USING iceberg

In [ ]:
%%sql
CREATE OR REPLACE TABLE db.sales_iceberg_analysis AS 
SELECT category, sum(price) as total_sales, count(*) as count_by_year, year(datetime) as year
FROM db.sales_iceberg 
GROUP BY category, year

事前に、本リポジトリの `sameple-data` ディレクトリ配下に存在する `data.json` を `s3://amzn-s3-demo-bucket/spark/streaming/input` 配下にアップロードします。アップロード手順については、[MinIO にファイルをアップロードする](https://github.com/murashitas/iceberg_book_handson#minio-にファイルをアップロードする)を参照してください。

### Structured Streaming 経由での Iceberg テーブルへの書き込み

In [ ]:
df = (
    spark.readStream.format("json")
        .schema("product_name STRING, price DECIMAL(10, 2), customer_id BIGINT, order_id STRING, datetime TIMESTAMP, category STRING")
        .option("inferSchema", "true")
        .option("cleanSource", "archive")
        .option("sourceArchiveDir", "s3a://amzn-s3-demo-bucket/spark/streaming/input-archives")
        .load("s3a://amzn-s3-demo-bucket/spark/streaming/input")
)

JSON ファイルのデータを読み込んで、Iceberg テーブルに書き込みます。

In [ ]:
sq = (
    df.writeStream 
        .format("iceberg")
        .outputMode("append")
        .option("checkpointLocation", "s3a://amzn-s3-demo-bucket/spark/streaming/checkpoints")
        .trigger(processingTime="5 seconds")
        .toTable("db.sales_iceberg")
)

数秒待って `db.sales_iceberg` にデータが書き込まれたか確認しましょう。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg LIMIT 10

書き込まれたことを確認したら、一度 Structured Streaming の処理を停止します。

In [ ]:
sq.stop()

### `sales_iceberg` テーブルからデータを読み出し、集計結果を `sales_iceberg_analysis` にストリーミング書き込みする
次に Iceberg テーブルから、ストリーミング読み出しを行い、別の Iceberg テーブルにストリーミング書き込みしてみましょう。

In [ ]:
df = spark.readStream.format("iceberg").load("db.sales_iceberg")

In [ ]:
from pyspark.sql.functions import year, sum, count
df_2 = (
    df.groupBy("category", year("datetime").alias("year"))
        .agg(
            sum("price").alias("total_sales"),
            count("*").alias("count_by_year")
        )
        .select("category", "total_sales", "count_by_year", "year")
)

ここでは集計結果を `complete` モードで書き出します。これにより、`sales_iceberg_analysis` には毎回新しい集計値が書き込まれます。過去の特定の時点における集計値を参照したい場合は、タイムトラベルクエリで確認します。

In [ ]:
sq = (
    df_2.writeStream 
        .format("iceberg")
        .outputMode("complete")
        .option("checkpointLocation", "s3a://amzn-s3-demo-bucket/spark/streaming/checkpoints-iceberg") # 本ノートブックでは別の checkpoint を指定しています。
        .trigger(processingTime="5 seconds")
        .toTable("db.sales_iceberg_analysis")
)

書き込みが完了したら、`sales_iceberg_analysis` を確認します。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_analysis ORDER BY category, year DESC

書き込まれたことを確認したら、再度 Structured Streaming の処理を停止します。

In [ ]:
sq.stop()

### 複雑な操作の実行
`sales_iceberg` に対してストリーミング Upsert 処理を行いながら、コンパクション処理も実行します。まずは、この時点における `sales_iceberg` テーブルにおけるレコード数を確認しておきましょう。`5000` と出力されるはずです。

In [ ]:
%%sql
SELECT count(*) FROM db.sales_iceberg

JSON ファイルをデータソースとして読み込みましょう。本リポジトリの `sameple-data` ディレクトリ配下に存在する `data-add.json` を `s3://amzn-s3-demo-bucket/spark/streaming/input` 配下にアップロードします。アップロード手順については、[MinIO にファイルをアップロードする](https://github.com/murashitas/iceberg_book_handson#minio-にファイルをアップロードする)を参照してください。

In [ ]:
df = (
    spark.readStream.format("json")
        .schema("product_name STRING, price DECIMAL(10, 2), customer_id BIGINT, order_id STRING, datetime TIMESTAMP, category STRING")
        .option("inferSchema", "true")
        .option("cleanSource", "archive")
        .option("sourceArchiveDir", "s3a://amzn-s3-demo-bucket/spark/streaming/input-archives")
        .load("s3a://amzn-s3-demo-bucket/spark/streaming/input")
)

In [ ]:
def process_batch(batch_df, batch_id):
    batch_df.createOrReplaceGlobalTempView("tmp")

    spark.sql("""
    MERGE INTO db.sales_iceberg t
    USING global_temp.tmp s
    ON t.order_id = s.order_id
    WHEN MATCHED THEN
        UPDATE SET t.product_name = s.product_name, t.price = s.price, t.datetime = s.datetime
    WHEN NOT MATCHED THEN INSERT *
    """)

    if batch_id % 20 == 0:
        spark.sql("""
        CALL system.rewrite_data_files (
            table => 'db.sales_iceberg', options => map('min-input-files', '5')
        )
        """)

In [ ]:
sq = (
    df.writeStream
        .foreachBatch(process_batch)
        .option("checkpointLocation", "s3a://amzn-s3-demo-bucket/spark/streaming/checkpoints-iceberg-merge") # 本ノートブックでは別の checkpoint を指定しています。
        .trigger(processingTime="10 seconds")
        .start()
)

以下のクエリを実行し、新たなレコードが書き込まれたことを確認したら、再度 Structured Streaming の処理を停止します。

In [ ]:
%%sql
SELECT count(*) FROM db.sales_iceberg

In [ ]:
sq.stop()